This is a [jupyter](https://jupyter.org/) notebook that contains text, equations, images and executable code in one document.

$
\newcommand{\dd}{\text{d}}
\newcommand{\e}{\operatorname{e}}
\newcommand{\Fc}{\underline F}
\newcommand{\Hc}{\underline H}
\newcommand{\Xc}{\underline X}
\newcommand{\Yc}{\underline Y}
\newcommand{\Zc}{\underline Z}
\newcommand{\xic}{\underline \xi}
\newcommand{\vc}{\underline v}
\newcommand{\xiv}{\pmb{\xi}}
\newcommand{\Psiv}{\pmb{\Psi}}
\newcommand{\xivc}{\underline \xiv}
\newcommand{\xv}{\mathbf{x}}
\newcommand{\jj}{\operatorname{j}}
$

# 4 Plane Wave Reflection
## 4.1 Free Planar Surface
* medium boundary with free surface at $y=0$
* _free_ means no restrictions for excitation normal to surface $\xi_y$
* stresses normal to surface vanish (boundary conditions):
\begin{gather}\label{eq:free_BC}
\sigma_y=0\\
\tau_{yx}=0\\
\tau_{yz}=0
\end{gather}

## 4.2 Wave Incidence at Planar Surface

![surface incidence](tikzfig/surface_incidence.svg "surface incidence")

* overall wave field may consist of both longitudinal and transversal waves (L-wave, T-wave)
* no dependence from $z$-coordinate: $k_z=0$
* two dimensional wave field $k^2=k_x^2+k_y^2$
* direction of plane wave propagation given by angle $\vartheta$:
\begin{equation}\label{eq:direction}
k_x=k \sin\vartheta\:,\: k_y=k \cos \vartheta
\end{equation}

**L-wave:**
* scalar potential 
\begin{equation}\label{eq:potentialL}
\Phi(\xv)=\Phi_0 \e^{-\jj k_{Lx}x}\e^{-\jj k_{Ly}y}
\end{equation}
with $k_{Lx}=k_L \sin\vartheta_L\:,\: k_{Ly}=k_L \cos \vartheta_L$ and $k_L=\frac\omega{c_L}$
* get displacement field from $\xivc_L(\xv)=\nabla\Phi(\xv)$: 

In [1]:
import sympy as sy
x,y, Phi0, thetaL = sy.symbols('x y Phi_0, vartheta_L')
xix,xiy = sy.symbols('xi_x xi_y')
kLx, kLy, kL = sy.symbols('k_Lx k_Ly k_L')
Phi = Phi0*sy.exp(-sy.I*(kLx*x+kLy*y))
def grad2D(f):
    return sy.Matrix([sy.diff(f,x),sy.diff(f,y)])
xiL = grad2D(Phi)
xiL = xiL.subs({kLx:kL*sy.sin(thetaL),kLy:kL*sy.cos(thetaL)})
xiL

Matrix([
[-I*Phi_0*k_L*exp(-I*(k_L*x*sin(vartheta_L) + k_L*y*cos(vartheta_L)))*sin(vartheta_L)],
[-I*Phi_0*k_L*exp(-I*(k_L*x*sin(vartheta_L) + k_L*y*cos(vartheta_L)))*cos(vartheta_L)]])

**T-Wave:**
* vector potential 
\begin{equation}\label{eq:potentialT}
\Psiv(\xv)=\Psiv_0 \e^{-\jj k_{Tx}x}\e^{-\jj k_{Ty}y}
\end{equation}
with $k_{Tx}=k_T \sin\vartheta_T\:,\: k_{Ty}=k_T \cos \vartheta_T$ and $k_T=\frac\omega{c_T}$
* get displacement field from $\xivc_T(\xv)=\nabla\times\Psiv(\xv)$
* because of no $z$-dependence of the field, $\Psiv$ always points into $z$-direction and its $x$- and $y$-components are zero
* replace $\Psiv$ with $\Psi_z$

In [2]:
import sympy as sy
Psi0, thetaT = sy.symbols('Psi_0, vartheta_T')
kTx, kTy, kT = sy.symbols('k_Tx k_Ty k_T')
Psiz = Psi0*sy.exp(-sy.I*(kTx*x+kTy*y))
# curl is very simple in 2D:
def curl2D(f):
    return sy.Matrix([sy.diff(f,y),-sy.diff(f,x)])
xiT = curl2D(Psiz)
xiT = xiT.subs({kTx:kT*sy.sin(thetaT),kTy:kT*sy.cos(thetaT)})
xiT

Matrix([
[-I*Psi_0*k_T*exp(-I*(k_T*x*sin(vartheta_T) + k_T*y*cos(vartheta_T)))*cos(vartheta_T)],
[ I*Psi_0*k_T*exp(-I*(k_T*x*sin(vartheta_T) + k_T*y*cos(vartheta_T)))*sin(vartheta_T)]])

* now let's plot the displacement vector field using a harmonic time dependency

In [3]:
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as pl
%matplotlib inline

mu_ = 0.3 # assume poisson ratio
cT = 1.0 # transverse speed of sound
cL = cT*np.sqrt(1+1/(1-2*mu_)) # longitudinal speed of sound

# make numeric functions and set kL and kT assuming omega=1
fxiL = sy.lambdify((x,y,thetaL),xiL.subs({kL:1/cL,Phi0:1.5})) 
fxiT = sy.lambdify((x,y,thetaT),xiT.subs({kT:1/cT,Psi0:1})) 

x0,y0 = np.meshgrid(np.linspace(-11.,12.,24),np.linspace(-13.,0,14))
color = np.zeros((13,23))

def plot_wave(kind,theta=45,t=0):
    t *= 2*np.pi
    f = {'L-Wave':fxiL,'T-Wave':fxiT}[kind]
    dx, dy = f(x0,y0,theta*np.pi/180+0j)*np.exp(1j*t)
    #print(dx)
    x = x0+0.5*dx[0].real
    y = y0+0.5*dy[0].real
    pl.pcolormesh(x,y,color,edgecolor='k',cmap='Greys')
    pl.ylim(-12.5,-0.5)
    pl.xlim(-10.5,11.5)
    pl.gca().set_axis_off()
    
ipw.interact(plot_wave,
             kind=['L-Wave','T-Wave'],
             theta=(0,180,15),
             t=(0,10,0.05));

interactive(children=(Dropdown(description='kind', options=('L-Wave', 'T-Wave'), value='L-Wave'), IntSlider(va…

## 4.3 Wave Reflection at Surface

### Superposition of three plane waves
![surface reflection](tikzfig/surface_reflection.svg "surface reflection")

* sound field consists of _three_ waves
* **incident L-wave** with $\vartheta_{iL}<90°$
* two reflected waves possible: L- and T-wave with $\vartheta_{rL}>90°$ and $\vartheta_{rT}>90°$
* at the surface all waves must have the same **trace wave length** in the $x$-direction
* all $k_x$ wavenumbers must be the same
\begin{equation}\label{eq:tracex}
k_x=k_{iLx}=k_{rLx}=k_{rTx}
\end{equation}
* compatibility of reflected and incident angles
\begin{equation}\label{eq:compat_anglesL}
k_{L}\sin\vartheta_{iL}=k_{L}\sin\vartheta_{rL}
\end{equation}
gives $\vartheta_{rL}=\pi-\vartheta_{iL}$ and
\begin{equation}\label{eq:compat_anglesT}
k_{L}\sin\vartheta_{iL}=k_{T}\sin\vartheta_{rT}
\end{equation}
and we solve:

In [4]:
mu = sy.symbols('mu')
eq = sy.Eq(kL*sy.sin(thetaL),kT*sy.sin(thetaT))
thetarT = sy.solve(eq,thetaT)[0]
display(thetarT)
thetarT.subs(kT,kL*sy.sqrt(1+1/(1-2*mu)))

pi - asin(k_L*sin(vartheta_L)/k_T)

pi - asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))

* this is a "refraction law" for the transverse wave with a "refraction index"
\begin{equation}\label{eq:refraction}
n = \frac{k_L}{k_T} = \frac{c_T}{c_L} = \left(1+\frac{1}{1-2\mu}\right)^{-\frac12}
\end{equation}

* reflected waves have amplitudes $\Phi_r$, $\Psi_r$
* reflection factors
\begin{equation}\label{eq:rfactors}
r_{LL}=\frac{\Phi_r}{\Phi_0},\:\:r_{LT}=\frac{\Psi_r}{\Phi_0}
\end{equation}
* lets calculate overall wave field from all three waves with some substitutions:

In [5]:
n, rLL, rLT = sy.symbols("n, r_LL r_LT")
thetarL = sy.pi - thetaL # angle for reflected L-wave
# incident L-wave
xiiL = xiL 
# reflected L-wave, substitute angle and reflection factor
xirL = xiL.subs({thetaL:thetarL,Phi0:rLL*Phi0})
# reflected T-wave, substitute reflection factor
xirT = xiT.subs({Psi0:rLT*Phi0})
# all three waves
xi = xiiL+xirL+xirT
# substitute T-wave number with refraction index
xi = xi.subs(kT,kL/n)
sy.simplify(xi)

Matrix([
[-I*Phi_0*k_L*(n*r_LL*exp(-I*k_L*(x*sin(vartheta_L) - y*cos(vartheta_L)))*sin(vartheta_L) + n*exp(-I*k_L*(x*sin(vartheta_L) + y*cos(vartheta_L)))*sin(vartheta_L) + r_LT*exp(-I*k_L*(x*sin(vartheta_T) + y*cos(vartheta_T))/n)*cos(vartheta_T))/n],
[ I*Phi_0*k_L*(n*r_LL*exp(I*k_L*(-x*sin(vartheta_L) + y*cos(vartheta_L)))*cos(vartheta_L) - n*exp(-I*k_L*(x*sin(vartheta_L) + y*cos(vartheta_L)))*cos(vartheta_L) + r_LT*exp(-I*k_L*(x*sin(vartheta_T) + y*cos(vartheta_T))/n)*sin(vartheta_T))/n]])

### Boundary conditions

* boundary conditions for shear stresses:
\begin{align}\label{eq:shearbc1}
\tau_{yx} (y=0)&=G \left(\frac{\partial \xi_x}{\partial y}+\frac{\partial \xi_y}{\partial x}\right) =0\\
\label{eq:shearbc2}
\tau_{yz} (y=0)&=G \left(\frac{\partial \xi_z}{\partial y}+\frac{\partial \xi_y}{\partial z}\right) =G \frac{\partial \xi_z}{\partial y} =0
\end{align}
where \eqref{eq:shearbc2} is fulfilled already because $\xi_z=0$ for incident L-wave
* boundary condition for normal stress:
\begin{multline}\label{eq:normalbc}
\sigma_y 
=2 G\left( \frac{\partial \xi_y}{\partial y} + \frac{\mu}{1-2\mu} \nabla\cdot\xiv\right)\\=2 G\left( \frac{\partial \xi_y}{\partial y} + \frac{\mu}{1-2\mu} \left(\frac{\partial \xi_x}{\partial x}+\frac{\partial \xi_y}{\partial y}\right)\right)=0
\end{multline}
* now substitute the three wave field into \eqref{eq:shearbc1} and \eqref{eq:normalbc} and solve for the reflection coefficients:

In [6]:
xix,xiy = xi # isolate x,y components
# boundary conditions for y=0
bcshear = sy.Eq(sy.diff(xix,y)+sy.diff(xiy,x),0).subs(y,0)
bcnormal = sy.Eq(sy.diff(xiy,y)
                 +mu/(1-2*mu)*(sy.diff(xix,x)+sy.diff(xiy,y)),0).subs(y,0)
solu = sy.solve((bcshear,bcnormal),(rLL,rLT))
display(sy.Eq(rLL,solu[rLL]))
display(sy.Eq(rLT,solu[rLT].subs(sy.sin(thetaT),n*sy.sin(thetaL))))

Eq(r_LL, (2*mu*cos(2*vartheta_L - 2*vartheta_T) - cos(2*vartheta_T) - cos(2*vartheta_L - 2*vartheta_T))/(-2*mu*cos(2*vartheta_L + 2*vartheta_T) + cos(2*vartheta_T) + cos(2*vartheta_L + 2*vartheta_T)))

Eq(r_LT, n**2*(2*mu*sin(4*vartheta_L) - 2*sin(2*vartheta_L) - sin(4*vartheta_L))/(-2*mu*cos(2*vartheta_L + 2*vartheta_T) + cos(2*vartheta_T) + cos(2*vartheta_L + 2*vartheta_T)))

* reflection coefficients depend on $\vartheta_L$ and $\mu$  ($\vartheta_{T}$ and $n$ can be substituted)

In [7]:
thetarT = thetarT.subs(kT,kL/n)
rLL1 = solu[rLL].simplify().subs(thetaT,thetarT)
rLL1 = rLL1.subs(n,1/sy.sqrt(1+1/(1-2*mu)))
display(sy.Eq(rLL,rLL1))
rLT1 = solu[rLT].subs(thetaT,thetarT)
rLT1 = rLT1.subs(n,1/sy.sqrt(1+1/(1-2*mu)))
display(sy.Eq(rLT,rLT1))

Eq(r_LL, (2*mu*cos(2*vartheta_L + 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) - cos(2*vartheta_L + 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) - cos(2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))))/(-2*mu*cos(2*vartheta_L - 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) + cos(2*vartheta_L - 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) + cos(2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu))))))

Eq(r_LT, (2*mu*sin(4*vartheta_L) - 2*sin(2*vartheta_L) - sin(4*vartheta_L))/((1 + 1/(1 - 2*mu))*(-2*mu*cos(2*vartheta_L - 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) + cos(2*vartheta_L - 2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))) + cos(2*asin(sin(vartheta_L)/sqrt(1 + 1/(1 - 2*mu)))))))

## 4.4 Energy Transport and Reflection Coefficient

### Intensity
* generic equation for energy transport - the intensity
\begin{equation}\label{eq:intensity}
I = c_g w
\end{equation}
where $c_g$ is the wave __group speed__ and $w$ is the **energy density** (energy per volume)
* energy density: temporal means of kinetic and potential energy are the same
* mean kinetic energy density is
\begin{equation}\label{eq:kinetic}
w_{kin}=\frac 12\rho \overline{{v}^2}=\frac 12\rho \omega^2\overline{\xi^2}=\frac 14\rho \omega^2\hat\xi^2
\end{equation}

**L-wave**:
* group speed is speed of sound $c_g=c_L$
* plane wave ($\xic_L$ in direction of travel):
\begin{gather}\label{eq:dispL}
\xic_L=-\jj k_L\Phi=- \jj \frac{\omega}{c_L}\Phi\\
w=w_{pot}+w_{kin}=2w_{kin}=\frac 12\rho \omega^2 k_L^2\Phi_0^2\\
\label{eq:intensityL}
I_L = \frac 12 c_L\rho \omega^2 k_L^2\Phi_0^2 = \frac 12 \rho \frac{\omega^4}{c_L}\Phi_0^2
\end{gather}

**T-wave**:
* group speed is speed of sound $c_g=c_T$
\begin{gather}
\label{eq:intensityT}
I_T = \frac 12 \rho \frac{\omega^4}{c_T}\Psi_{0z}^2
\end{gather}


### Reflection Coefficients

* ratio of reflected and incident power per boundary length 
\begin{equation}\label{eq:rcoeffL}
\rho_{LL} = \frac{P_{rL}}{P_{iL}}=\frac{I_{rL}|\cos\vartheta_{rL}|}{I_{iL}|\cos\vartheta_L|}=r^2_{LL}
\end{equation}
_note the difference between density $\rho$ (without index) and reflection coefficient $\rho_{LL}$ (with index)_
\begin{multline}\label{eq:rcoeffT}
\rho_{LT}
=\frac{P_{rT}}{P_{iL}}
=\frac{I_{rL}|\cos\vartheta_{rT}|}{I_{iL}|\cos\vartheta_L|}
=r^2_{LT}\frac{c_L|\cos\vartheta_{rT}|}{c_T|\cos\vartheta_L|}
=\frac{r^2_{LT}}{n}\frac{|\cos\vartheta_{rT}|}{|\cos\vartheta_L|}
\end{multline}
* let's see how the reflection coefficients depend on the incidence angle and on $\mu$:

In [8]:
th = np.linspace(0,np.pi/2,46)
rLL_ = sy.lambdify((thetaL,mu),rLL1)
rLT_ = sy.lambdify((thetaL,mu),rLT1)
thrT_ = sy.lambdify((thetaL,mu),thetarT.subs(n,1/sy.sqrt(1+1/(1-2*mu))))
def plotrcoeffs(mu_=0.3):
    n_ = np.sqrt((1-2*mu_)/(2-2*mu_))
    pl.plot(th*180/np.pi,rLL_(th,mu_)**2,label=r'$\rho_{LL}$')
    pl.plot(th*180/np.pi,
            rLT_(th,mu_)**2*abs(np.cos(thrT_(th,mu_)))/np.cos(th)/n_,
            label=r'$\rho_{LT}$')
    pl.xlabel(r'$\vartheta_L$')
    pl.legend();
    
ipw.interact(plotrcoeffs,mu_=(0,0.499,0.05));    

interactive(children=(FloatSlider(value=0.3, description='mu_', max=0.499, step=0.05), Output()), _dom_classes…

## 4.5 Total Reflection 

* for incident T-wave: 
\begin{equation}
\rho_{TT}(\vartheta_L) = \rho_{LL}(\vartheta_L)\\
\rho_{TL}(\vartheta_L) = \rho_{LT}(\vartheta_L)
\end{equation}
* angles $\vartheta_L$,$\vartheta_{rL}$ and $\vartheta_T$ are coupled
\begin{gather}
\sin\vartheta_T=n \sin\vartheta_L\\
\vartheta_{rL} = \pi - \arcsin(\frac{\sin{\vartheta_T}}{n})
\end{gather}
* total reflection $\rho_{TL}$ occurs if $\vartheta_T$ is too large
\begin{equation}
\vartheta_T>\arcsin n
\end{equation}



In [9]:
def plotthetas(mu_=0.3):
    thrL = 180-th*180/np.pi
    thT = 180-thrT_(th,mu_)*180/np.pi
    pl.plot(thT,thrL)
    pl.ylabel(r'$\vartheta_{rL}$')
    pl.xlabel(r'$\vartheta_T$')

ipw.interact(plotthetas,mu_=(0,0.499,0.05));    

interactive(children=(FloatSlider(value=0.3, description='mu_', max=0.499, step=0.05), Output()), _dom_classes…

* what happens to L-wave in case of total reflection
* wave numbers $k_{Tx}=k_{Lx}>k_L$
\begin{equation}\label{eq:}
k_{Ly}=\sqrt{k_L^2-k_{Tx}^2}
\end{equation}
becomes imaginary
* **near field** (no energy transport in $y$-direction)
* surface 'wave'

In [10]:
xiL = grad2D(Phi)
xiL = xiL.subs({kLx:kT**2 * sy.sin(thetaT),
                kLy:sy.sqrt(kL**2-kT**2 * sy.sin(thetaT)**2)})
# make numeric functions and set kL and kT assuming omega=1
fxiL = sy.lambdify((x,y,thetaT),xiL.subs({kL:1/cL,kT:1/cT,Phi0:1.5})) 

def plot_swave(theta=45,t=0):
    t *= 2*np.pi
    dx, dy = fxiL(x0,y0,theta*np.pi/180+0j)*np.exp(1j*t)
    x = x0+0.5*dx[0].real
    y = y0+0.5*dy[0].real
    pl.pcolormesh(x,y,color,edgecolor='k',cmap='Greys')
    pl.ylim(-12.5,1)
    pl.xlim(-10.5,11.5)
    pl.gca().set_axis_off()
    
ipw.interact(plot_swave,
             #kind=['L-Wave','T-Wave'],
             theta=(0,90,5),
             t=(0,10,0.05));

interactive(children=(IntSlider(value=45, description='theta', max=90, step=5), FloatSlider(value=0.0, descrip…

## 3.6 Rayleigh Waves

* surface waves can occur also for transverse component, if $k_{Tx}>k_T$
* if $k_{Tx}>k_T$ then also $k_{Lx}>k_L$
* for a certain $k_x$, combined T-wave and L-wave give a surface wave which propagates without any incident waves (free wave)
* Rayleigh wave propagates with with speeds $c_R$ a little bit smaller than $c_T$
* $c_R$ is given as a solution of a third order equation (many pages long)
* example for $\mu=0.3$:
\begin{equation}\label{eq:rayleigh}
c_R \approx 0.93 c_T 
\end{equation}
* important for earthquakes and for ultrasonic waves in solids

#### ----------------------------------------
* plane wave reflection can produce one or two reflected waves
* near fields are possible 
* surface waves
* special case Rayleigh wave: free surface wave 
* next topic: structure-borne sound in plates (two parallel surfaces)

#### License

This notebook is an [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use it for your own purposes. The text and the images are licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), and any code under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: Ennes Sarradj, Structure-borne sound lecture notes, 2019.

In [11]:
# this is just for custom formatting
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()